<a href="https://colab.research.google.com/github/sandip01112004/MediScan/blob/main/AI_MediScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive to access the uploaded file
drive.mount('/content/drive')

# Path to the uploaded zip file
zip_path = '/content/drive/My Drive/eye_disease_dataset.zip'
extract_path = '/content/eye_disease_dataset'

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check if the extraction was successful
os.listdir(extract_path)


Mounted at /content/drive


['cataract', 'diabetic_retinopathy', 'normal', 'glaucoma']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [ ]:
base_dir = '/content/eye_disease_dataset'
categories = ['cataract', 'diabetic_retinopathy', 'normal', 'glaucoma']


In [ ]:
img_size = (224, 224)
data = []
labels = []

def load_and_preprocess_image(filepath, label):
    image = cv2.imread(filepath)
    if image is not None:
        image = cv2.resize(image, img_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        data.append(image)
        labels.append(label)

for category in categories:
    category_path = os.path.join(base_dir, category)
    label = categories.index(category)
    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        load_and_preprocess_image(img_path, label)

data = np.array(data)
labels = np.array(labels)

print(f"Loaded {data.shape[0]} images.")


Loaded 4217 images.


In [ ]:
labels = to_categorical(labels, num_classes=len(categories))


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Normalize the data
X_train = X_train / 255.0
X_test = X_test / 255.0


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
